<a href="https://colab.research.google.com/github/tanyaryabov/ML/blob/master/Content_Based_Netflix_Recommender__euc_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive

Mounted at /content/drive
MyDrive  Shareddrives


In [ ]:
DATA_PATH= '/content/drive/MyDrive/netflix_titles.csv'

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Load Dataset
df = pd.read_csv('/content/drive/MyDrive/netflix_titles.csv')
print(df.shape)
df.head()

(7787, 12)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [ ]:
# TF-IDF weighs a term’s frequency (TF) and its inverse document frequency (IDF). 
# The TF (term frequency) of a word is the frequency of a word (number of times it appears) in a 1 document.
# The IDF (inverse document frequency) of a word is the measure of how significant that term is in the whole corpus (1 million documents).

from sklearn.feature_extraction.text import TfidfVectorizer
#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
df['description'] = df['description'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df['description'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(7787, 17905)

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
# Compute the euclidian distance matrix
euc_dist = euclidean_distances(tfidf_matrix, tfidf_matrix)
euc_dist.shape
euc_dist[1]

array([1.41421356, 0.        , 1.41421356, ..., 1.34461865, 1.41421356,
       1.41421356])

In [ ]:
#Construct a map of indixces and movie titles
indixes = pd.Series(df.index, index=df['title']).drop_duplicates()
indixes[:10]

title
3%       0
7:19     1
23:59    2
9        3
21       4
46       5
122      6
187      7
706      8
1920     9
dtype: int64

In [ ]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, euc_dist=euc_dist):
    # Get the index of the movie that matches the title
    idx = indixes[title]

    # Get the pairwsie similarity scores of all movies with that movie
    euc_scores = list(enumerate(euc_dist[idx]))

    # Sort the movies based on the similarity scores
    euc_scores = sorted(euc_scores, key=lambda x: x[1], reverse=False)
   

    # Get the scores of the 10 most similar movies
    euc_scores = euc_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in euc_scores]

    # Return the top 10 most similar movies
    return df['title'].iloc[movie_indices]

In [ ]:
get_recommendations('Bridgerton')

3098                     Jack Taylor
7572            What's Up With Love?
2509                Graceful Friends
6540                      The L Word
3007                         Inmates
472     American Pie 9: Girls' Rules
524           Angry Indian Goddesses
712               Bana Masal Anlatma
4048            Merry Happy Whatever
3576    LEGENDS OF THE HIDDEN TEMPLE
Name: title, dtype: object

In [ ]:
get_recommendations('LEGENDS OF THE HIDDEN TEMPLE')

7303                                 Ultimate Beastmaster
3593    LEGO Ninjago: Masters of Spinjitzu: Day of the...
7304                          Ultimate Beastmaster México
2883                                           Hyperdrive
377                                                 Alelí
2728                                    High End Yaariyan
3421                                           Knightfall
905                                           Big Miracle
6313                                          The Dreamer
96                                         3 Days to Kill
Name: title, dtype: object